In [2]:
import pandas as pd 
import networkx as nx 
from tqdm import tqdm 
tqdm.pandas()
import matplotlib.pyplot as plt
from statistics import mean
from tqdm import tqdm 
from datetime import datetime 
import csv 
from utils_graph import *
from operator import add
import math 

In [3]:
# type = 'MultiDiGraph'
# type = 'undirected'
type = 'MultiGraph'
GC = build_graph_from_csv('../data/cpnet47rel/conceptnet.en.csv', type)
GS = build_graph_from_csv('../data/swow/conceptnet.en.csv', type)

In [4]:
def get_sub_graph(S,T, direction):
    '''
    1) the shared sub-graph, including their shared edges and their relations 
    2) the unique sub-graph 

    S - T 
    return: a sub-graph of S 
    '''
    R=S.copy()

    # compute shared nodes
    R.remove_nodes_from(n for n in S if n not in T)
    N_shared_nodes = R.order() 
    
    # compute shared edges 
    rm_list = []
    for (u,v, (rel)) in tqdm(R.edges.data()):
        if not T.has_edge(u,v):
            rm_list.append((u,v))
    R.remove_edges_from(x for x in rm_list)

    #compute the unique graph 
    H = S.copy()
    for (u,v, (rel)) in tqdm(H.edges.data()):
        if R.has_edge(u,v):
            rm_list.append((u,v))
    H.remove_edges_from(x for x in rm_list)

    return R, H 

In [5]:
def print_edges(G):
    for (u,v, (rel)) in tqdm(G.edges.data()):
        rel = rel['rel']
        print(u,v, rel)

In [6]:
Gs = nx.MultiDiGraph()
seed='camping'
GC1 = nx.ego_graph(GC, seed)
GS1 = nx.ego_graph(GS, seed)

H1, H2  = get_sub_graph(GC1, GS1, direction='CN->SW')
S1, S2  = get_sub_graph(GS1, GC1, direction='SW->CN')
# selected_rels = ('', 'hasproperty', 'mannerof', 'formof', 'derivedfrom')

# nx.draw(H1, with_labels=True)
# plt.show()
# nx.draw(H2,with_labels=True)


100%|██████████| 3770/3770 [00:00<00:00, 887646.01it/s]


In [13]:

print_edges(H1)
print("#"*30)
print_edges(S1)

100%|██████████| 42/42 [00:00<00:00, 3399.67it/s]fire camp relatedto
fire camp relatedto
fire campfire isa
fire campfire relatedto
fire campfire relatedto
fire camping relatedto
knapsack camping relatedto
camp campfire relatedto
camp lake relatedto
camp camping formof
camp lantern relatedto
camp fun relatedto
camp tent relatedto
camp tent relatedto
camp tent relatedto
camp tent relatedto
camp tent synonym
campfire camping relatedto
enjoyable camping hasproperty
enjoyable fun hasproperty
enjoyable fun relatedto
enjoyable fun similarto
enjoyable fun synonym
lake camping relatedto
lake fun relatedto
lake fun usedfor
camping fun hasproperty
camping tent relatedto
camping tent relatedto
camping tent usedfor
camping canteen relatedto
camping lantern relatedto
camping trailer relatedto
camping tenting synonym
##############################
knapsack camping forwardassociated
campfire camping forwardassociated
campfire camping bidirectionalassociated
campfire camping forwardassociated
campfire 

In [ ]:
# selected example: 
coffee cup atlocation
coffee hot hasproperty
tea beverage isa
coffee bitter hasproperty
coffee bean relatedto

coffee bean forwardassociated
coffee bean bidirectionalassociated

coffee bitter forwardassociated
coffee bitter bidirectionalassociated

In [14]:
print_edges(H2)
print("#"*30)
print_edges(S2)

  7%|▋         | 263/3718 [00:00<00:01, 2608.15it/s]noncamping camping relatedto
outdoor_activity camping isa
outdoor_activity camp relatedto
anticamping camping relatedto
vamping camping relatedto
glamper glamping etymologicallyrelatedto
glamper glamping relatedto
glamper glamping relatedto
glamper camping relatedto
includes_tents camping capableof
billycan camping relatedto
camping_ground camping derivedfrom
camping_ground camping_site similarto
camping_ground camping_site synonym
camper_shell camping relatedto
knapsack backpacking relatedto
very_fun camping hasproperty
favorite_activity_of_boy_scouts camping isa
done_at_campsite camping receivesaction
activity camp relatedto
activity camping isa
activity fun isa
activity fun relatedto
activity canoe_camping relatedto
what_do_in_camps camping isa
very_trying camping hasproperty
bivouacking camping synonym
campcraft camping relatedto
camping_site camping derivedfrom
camp sleeping_outside relatedto
camp encampment relatedto
camp encamp

In [14]:
# selected tuples 
#  coffee office atlocation

# input (u,v), output (relation)
def verify_edges(selected_nbs, T):
    for nb in selected_nbs:
        edge = T.edges([seed, nb ], data=True)
        for x in edge:
            h,t, rel = x 
            if seed in x and nb in x: 
                print(seed, nb, rel['rel'])

selected_nbs= ['nature', 'ground', 'trail', 'adventure', 'woods', 'family', 'hike', 'marshmallow']
verify_edges(selected_nbs, GS)


camping nature forwardassociated
camping nature bidirectionalassociated
camping nature forwardassociated
camping nature bidirectionalassociated
camping ground forwardassociated
camping ground bidirectionalassociated
camping ground forwardassociated
camping ground bidirectionalassociated
camping trail forwardassociated
camping adventure forwardassociated
camping adventure bidirectionalassociated
camping adventure forwardassociated
camping adventure bidirectionalassociated
camping woods forwardassociated
camping woods bidirectionalassociated
camping woods forwardassociated
camping woods bidirectionalassociated
camping family forwardassociated
camping hike forwardassociated
camping hike bidirectionalassociated
camping hike forwardassociated
camping hike bidirectionalassociated
camping marshmallow forwardassociated
camping marshmallow bidirectionalassociated
camping marshmallow forwardassociated
camping marshmallow bidirectionalassociated


In [12]:

seed = 'stunt'
sw_neihgbors=set()


Gs = nx.MultiDiGraph()
for k,v in GS[seed].items():
    # print(k,v['forwardassociated']['rel'])
    
    sw_neihgbors.add(k)
    for k1, v1 in v.items():
        Gs.add_edge(seed,k,rel=k1,)
        print(seed, k,k1,)
sw_cn=[]
for k,v in GC[seed].items():
    for k1,v1 in v.items():
        if k in sw_neihgbors:
            sw_cn.append((seed,k,k1))
        if k1 in ['derivedfrom', 'antonym', 'hascontext', 'relatedto','distinctfrom', 'formof']:
            print(k,k1)
                     
print("#######")
shared_neighbor=set()
if len(sw_cn)>0:
    for x in sw_cn:
        print(x)
        shared_neighbor.add(x[1])

print("#######SW only######")

stunt slow 0
stunt halt 0
stunt growth 0
stunt man 0
stunt driver 0
stunt explosion 0
stunt trick 0
stunt trick 1
stunt motorbike 0
stunt flair 0
stunt act 0
stunt crazy 0
stunt cunning 0
stunt dangerous 0
stunt dangerous 1
stunt hot_dog 0
stunt reckless 0
stunt movies 0
stunt prank 0
stunt prank 1
stunt antic 0
stunt double 0
stunt motorcycle 0
stunt fire 0
stunt hard 0
stunt fast 0
stunt make 0
stunt restrict 0
stunt showcase 0
stunt actor 0
stunt movie 0
stunt bike 0
stunt woman 0
stunt athletic 0
stunt bruises 0
stunt risk 0
stunt fall 0
stunt gimmick 0
stunt agile 0
stunt publicity 0
stunt publicity 1
stunt men 0
stunt circus 0
stunt antics 0
stunt silly 0
stunt evil_knee_val 0
stunt bad_idea 0
stunt car 0
stunt danger 0
stunt injury 0
stunt bicycle 0
stunt trapeze 0
stunt action 0
stunt hi_jinx 0
stunt pretend 0
stunt trade 0
stunt act_of_daring 0
stunt jackass 0
stunt ploy 0
stunt risky 0
stunt stet 0
stunt runt 0
stunt jump 0
stunt dare-devil 0
stunt event 0
stunt helmet 0
stun

In [23]:
def get_ent_rel_entropy(G):
    #goal: generate probibility for each relation
    #count the number of each relation appears
    rel2count = {}
    ent2count = {}

    Nedge = G.size()
    for (src, tgt, rel) in tqdm(G.edges.data(data=True), total=Nedge, desc='computing entropy'):
        rel = rel['rel']
        # print(src, tgt, rel)
        if rel not in rel2count:
            rel2count[rel]=1
        else:
            rel2count[rel]=+1

        if src not in ent2count:
            ent2count[src]=1
        else:
            ent2count[src]=+1

        if tgt not in ent2count:
            ent2count[tgt]=1
        else:
            ent2count[tgt]=+1

    
    ent2prob = {k : (v/Nedge) for k,v in tqdm(ent2count.items())}
    ent2entropy = {k : -v*math.log(v) for k,v in tqdm(ent2prob.items())}
    ent_entropy = sum(v for k, v in tqdm(ent2entropy.items()))
    # print("Entity Entropy : {}".format(ent_entropy))

    # rel2prob = {k : (v/graph.edgeCount) for k,v in rel2count.items()}
    # rel2entropy = {k : -v*math.log(v) for k,v in rel2prob.items()}
    # rel_entropy = sum(v for k, v in rel2entropy.items())
    # print("Relation Entropy : {}".format(rel_entropy))
    return ent_entropy

In [26]:
df = pd.DataFrame(data= {})
relation_num = {"ConceptNet":47, "SWOW":2}

directed = True if type in ('MultiDiGraph') else False 
def get_statistics(G, kg_name):
    density = nx.density(G)
    
    avg_degrees = average_degree(G, directed=directed)
    # node_entropy = get_node_entropy(G) 
    
    node_entropy = get_ent_rel_entropy(G)
    columns = ['#Triples', '#Nodes', '#Rels', 'Density', 'Degree', 'HN']
    df = pd.DataFrame(data={
        'KG': kg_name, 
        '#Triples': G.size(),
        '#Nodes': G.order(),
        '#Rels': relation_num.get(kg_name), 
        'Density': density,
        'Degree': avg_degrees,
        'HN': node_entropy
    }, index = [0])
    # print(df)
    return df 

df1=get_statistics(GC, "ConceptNet")
df2=get_statistics(GS, "SWOW")
df = pd.concat([df1, df2])
print(df) 


100%|██████████| 124626/124626 [00:00<00:00, 2389661.47it/s]
   KG  #Triples   #Nodes #Rels   Density     Degree        HN
0  CN   3009636  1080759  None  0.000003   2.784743  5.356807
0  SW   1593564   124626  None  0.000103  12.786770  1.116895


In [ ]:
#
# MultiDiGraph graph 
   KG  #Triples   #Nodes #Rels   Density     Degree        HN
0  CN   3009636  1080759  47  0.000003   2.784743  5.356807
0  SW   1593564   124626  2   0.000103  12.786770  1.116895


# Undirected  graph 
   KG #Triples   #Nodes  #Rels   Density     Degree        HN
0  CN  2587451  1080759     47  0.000004   4.788211  6.167725
0  SW  1280059   124626      2  0.000165  20.542407  1.369111

In [24]:
def get_node_entropy(G):
    '''
    210608: a simper version of computing node entropy, but the results are inconsitent wiht previous one, using the old one

    '''
    degrees =  [val for (node, val) in G.degree()]

    E = G.size()
    ent2prob = list(map(lambda x: x/E, degrees))
    ent2entropy = list(map(lambda x: -x*math.log(x), ent2prob))
    ent_entropy = sum(ent2entropy)
    print(ent_entropy)
    return ent_entropy 

In [18]:

# G = GS 
# out_degrees = [val for (node, val) in G.out_degree()]
# in_degrees =  [val for (node, val) in G.in_degree()]
# degrees = list( map(add, out_degrees, in_degrees) )

    # ent2prob = {k : (v/) for k,v in ent2count.items()}
    # ent2entropy = {k : -v*math.log(v) for k,v in ent2prob.items()}
    # self.ent_entropy = sum(v for k, v in ent2entropy.items())

18.18113300210889
